In [13]:
import pandas as pd
import pulp
import math

In [32]:
demand_df = pd.read_csv("data/demand.csv")
vehicles_df = pd.read_csv("min_ce_combined.csv")
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")
current_fleet = pd.read_csv("current_fleet.csv")

In [33]:
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         256 non-null    int64 
 1   Size         256 non-null    object
 2   Distance     256 non-null    object
 3   Demand (km)  256 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 8.1+ KB


In [34]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Allocation               256 non-null    object 
 1   Operating Year           256 non-null    int64  
 2   Size                     256 non-null    object 
 3   Distance_demand          256 non-null    object 
 4   Demand (km)              256 non-null    int64  
 5   ID                       256 non-null    object 
 6   Vehicle                  256 non-null    object 
 7   Available Year           256 non-null    int64  
 8   Cost ($)                 256 non-null    int64  
 9   Yearly range (km)        256 non-null    int64  
 10  Distance_vehicle         256 non-null    object 
 11  Fuel                     256 non-null    object 
 12  carbon_emissions_per_km  256 non-null    float64
 13  insurance_cost           256 non-null    float64
 14  maintenance_cost         2

In [35]:
current_fleet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Operating Year           0 non-null      object
 1   Size                     0 non-null      object
 2   Distance_demand          0 non-null      object
 3   Demand (km)              0 non-null      object
 4   ID                       0 non-null      object
 5   Vehicle                  0 non-null      object
 6   Available Year           0 non-null      object
 7   Cost ($)                 0 non-null      object
 8   Yearly range (km)        0 non-null      object
 9   Distance_vehicle         0 non-null      object
 10  Fuel                     0 non-null      object
 11  carbon_emissions_per_km  0 non-null      object
 12  insurance_cost           0 non-null      object
 13  maintenance_cost         0 non-null      object
 14  fuel_costs_per_km        0 non-null      object
 15  T

In [36]:
carbon_limits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Year                    16 non-null     int64
 1   Carbon emission CO2/kg  16 non-null     int64
dtypes: int64(2)
memory usage: 388.0 bytes


In [37]:
prob = pulp.LpProblem("Fleet_Optimization", pulp.LpMinimize)

In [38]:
vehicles_df

,Allocation,Operating Year,Size,Distance_demand,Demand (km),ID,Vehicle,Available Year,Cost ($),Yearly range (km),Distance_vehicle,Fuel,carbon_emissions_per_km,insurance_cost,maintenance_cost,fuel_costs_per_km
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,2023,187000,102000,D1,Electricity,0.000000,9350.00,1870.00,0.171278
1,LNG_S1_2023,2023,S1,D2,2597094,LNG_S1_2023,LNG,2023,100000,102000,D4,BioLNG,0.062503,5000.00,1000.00,0.179663
2,LNG_S1_2023,2023,S1,D3,3292011,LNG_S1_2023,LNG,2023,100000,102000,D4,BioLNG,0.062503,5000.00,1000.00,0.179663
3,LNG_S1_2023,2023,S1,D4,414315,LNG_S1_2023,LNG,2023,100000,102000,D4,BioLNG,0.062503,5000.00,1000.00,0.179663
4,BEV_S2_2023,2023,S2,D1,995694,BEV_S2_2023,BEV,2023,272000,106000,D1,Electricity,0.000000,13600.00,2720.00,0.171998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,BEV_S3_2038,2038,S3,D4,306910,BEV_S3_2038,BEV,2038,215849,73000,D4,Electricity,0.000000,10792.45,2158.49,0.089311
252,BEV_S4_2038,2038,S4,D1,23437,BEV_S4_2038,BEV,2038,276287,118000,D4,Electricity,0.000000,13814.35,2762.87,0.088993
253,BEV_S4_2038,2038,S4,D2,1148555,BEV_S4_2038,BEV,2038,276287,118000,D4,Electricity,0.000000,13814.35,2762.87,0.088993
254,BEV_S4_2038,2038,S4,D3,161386,BEV_S4_2038,BEV,2038,276287,118000,D4,Electricity,0.000000,13814.35,2762.87,0.088993


In [39]:
fleet_results = {}

for year in range(2023, 2039):
    unique_demand_combination_for_year = demand_df.groupby(['Size', 'Distance'])
    carbon_limit = carbon_limits_df[carbon_limits_df['Year'] == year]['Carbon emission CO2/kg'].values[0]
    print(year)
    year_df = pd.DataFrame(columns=[
        'Operating Year', 'Size', 'Distance', 'Demand (km)', 'ID', 'Num_Vehicles', 
        'Type', 'Fuel', 'Available Year', 'Cost ($)', 'Yearly range (km)',
        'Distance_vehicle', 'carbon_emissions_per_km', 'insurance_cost', 
        'maintenance_cost', 'fuel_costs_per_km', 'Topsis_Score', 'Rank'
    ])
    total_emissions = []
    current_fleet = []
    for (size, distance), group in unique_demand_combination_for_year:
        row = demand_df[(demand_df['Size'] == size) & 
                   (demand_df['Distance'] == distance) & 
                   (demand_df['Year'] == year)]
        
        demand = row['Demand (km)'].values[0] if not row.empty else None
        available_vehicles = []
        row = vehicles_df[(vehicles_df['Available Year'] == year) & 
                        (vehicles_df['Distance_demand'] == distance) & 
                        (vehicles_df['Size'] == size)]
        if not row.empty:
            available_vehicles.append(row['ID'].values[0])

        if len(current_fleet) > 0:
            row =  current_fleet[(current_fleet['Available Year'] == year) & 
                        (current_fleet['Distance_demand'] == distance) & 
                        (current_fleet['Size'] == size)]
            if not row.empty:
                available_vehicles.append(row['ID'].values[0])
        
        prob = pulp.LpProblem(f"Fleet_Optimization_{year}_{size}_{distance}", pulp.LpMinimize)
        
        for id in available_vehicles:
            row = vehicles_df[(vehicles_df['ID'] == id)]
            yearly_range = row['Yearly range (km)'].values[0] if not row.empty else None
            ub = math.ceil(demand/yearly_range)
            vehicle_vars = {id: pulp.LpVariable(f"num_vehicles_{id}", lowBound=0, cat='Integer')}
        
        # Objective: Minimize total carbon emissions
        for id in available_vehicles:
            prob += pulp.lpSum(vehicle_vars[id] * row['carbon_emissions_per_km'].values[0] * row['Yearly range (km)'].values[0])
            
            # Constraint: Meet the demand
            prob += pulp.lpSum(vehicle_vars[id] * row['Yearly range (km)'].values[0]) >= demand
            
        prob.solve()
        
        if vehicle_vars[id].varValue > 0:
            fleet_decision = {
                'Operating Year': year,
                'Size': size,
                'Distance': distance,
                'Demand (km)': demand,
                'ID': id,
                'Num_Vehicles': vehicle_vars[id].varValue,
                'Type': 'Buy' if year == 2023 else 'Use',
                'Fuel': row['Fuel'].values[0],
                'Available Year': row['Available Year'].values[0],
                'Cost ($)': row['Cost ($)'].values[0],
                'Yearly range (km)': row['Yearly range (km)'].values[0],
                'Distance_vehicle': row['Distance_vehicle'].values[0],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'].values[0],
                'insurance_cost': row['insurance_cost'].values[0],
                'maintenance_cost': row['maintenance_cost'].values[0],
                'fuel_costs_per_km': row['fuel_costs_per_km'].values[0]
            }
            
            year_df = pd.concat([year_df, pd.DataFrame([fleet_decision])], ignore_index=True)
            year_df.to_csv(f"fleet_{year}.csv")
    fleet_results[year] = year_df
print("Fleet optimization complete.")


2023


C:\Users\yasht\AppData\Local\Temp\ipykernel_20736\738600757.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  year_df = pd.concat([year_df, pd.DataFrame([fleet_decision])], ignore_index=True)


2024


C:\Users\yasht\AppData\Local\Temp\ipykernel_20736\738600757.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  year_df = pd.concat([year_df, pd.DataFrame([fleet_decision])], ignore_index=True)


2025


...

2037


C:\Users\yasht\AppData\Local\Temp\ipykernel_20736\738600757.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  year_df = pd.concat([year_df, pd.DataFrame([fleet_decision])], ignore_index=True)


2038


C:\Users\yasht\AppData\Local\Temp\ipykernel_20736\738600757.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  year_df = pd.concat([year_df, pd.DataFrame([fleet_decision])], ignore_index=True)


Fleet optimization complete.


In [41]:
for year in range(2023, 2039):
    df = pd.read_csv(f"fleet_{year}.csv")
    df['sum'] = df.apply(lambda row: row['Num_Vehicles'] * (row['carbon_emissions_per_km'] * (row['Demand (km)']/row['Num_Vehicles'])), axis=1)

    Total = df['sum'].sum()
    print(f"{year}: {Total}")

2023: 821042.1362585184
2024: 839098.4741546062
2025: 862826.4797781922
2026: 396506.1922270984
2027: 396716.1120826815
2028: 400378.72232094937
2029: 50894.47228674816
2030: 52385.65815110709
2031: 54194.35233792967
2032: 0.0
2033: 0.0
2034: 0.0
2035: 0.0
2036: 0.0
2037: 0.0
2038: 0.0
